# Conduct trimming of raw reads

Generate list of all the .fq files. Assumes we are the directory with the raw reads

In [ ]:
ls -d $PWD/*.fq.gz > fqFiles

Generate multiple trimmed.sh files per sample 

from os import listdir as ls
from os import path as op
import os,sys
import os
import shutil

In [ ]:
home = "/home/menonm2/eckertlab/Mitra/chapter3/rawData/128.120.88.251/H202SC19100774/raw_data"
f=open("fqFiles")
line_f=f.readlines()

Loop through samples with a step of 2 to get both F and R reads

In [ ]:

for i in range(0,len(line_f),2):
	F = str.strip(line_f[i]) #STRIP TO REMOVE THE WHITE SPACE AND ALL OTHER CHARACTERS AT EITHER END OF LINE
	R = str.strip(line_f[i+1])


#STEP 1
	a1F=F.split("/")[-1]
	a1R=R.split("/")[-1]
#STEP 2 
	a2=a1F.split("_")[1]
	
#STEP 3: make a directory
	 dest1 = op.join(home,"Trim_%s" % a2)
	 os.makedirs(dest1)
#STEP4: COPY FILES OVER (not needed currently)
	
	dest2_F = op.join(dest1,a1F)
	dest2_R = op.join(dest1,a1R)
	#shutil.copyfile(F,dest2_F)
	#shutil.copyfile(R,dest2_R)
	
	
#STEP5: create shell file


text = '''#!/usr/bin/env bash
#$ -N trim%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -pe smp 20
#$ -cwd
#$ -e ./
#$ -o ./
#$ -l mem_free=50G

trim_galore --fastqc -q 20 -a AATGATACGGCGACCACCGAGA -a2 CAAGCAGAAGACGGCATACG -stringency 5 --length 20 --paired %s %s --trim-n -j 4 --retain_unpaired

#change names of the trimmed files
 trim1F=$(sed 's/.fq.gz/_val_1.fq.gz/g' <<<"%s")
 trim1R=$(sed 's/.fq.gz/_val_2.fq.gz/g' <<<"%s")

#now remove polyAs from the previous trimmed files
trim_galore --polyA --paired $trim1F $trim1R -j 4

	''' % (a2,F,R,dest2_F,dest2_R)

	filE = op.join(dest1,"Trim%s.sh" % a2)
	with open (filE,'w') as o:
    	o.write("%s" % text)


Now actually submit all the trimming jobs

In [ ]:
for D in ./Trim*; do
	#echo "$D"
	cd "$D"
	qsub Trim*
	cd ../
done

# Assemble transcriptome

*Here we are following guidlines for assembling species with high genetic diversity, we obtain several transcriptomes and later reduce the redudancy*

We will do this for a subset of the samples at BS and WP (both gardens, to get a good representation of the transcriptome)

In [ ]:
t=open("trimmed_subset") #this is trimmed fastq names with full path
line_t=t.readlines()


Loop through the subset of trimmed directories

In [ ]:
for i in range(0,len(line_t),2):
    
	F = str.strip(line_f[i]) #STRIP TO REMOVE THE WHITE SPACE AND ALL OTHER CHARACTERS AT EITHER END OF LINE
	R = str.strip(line_f[i+1])

	dir=F.split("/")[-2]

text = '''#!/usr/bin/env bash
#$ -N assemble%s
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -pe smp 20
#$ -cwd
#$ -e ./
#$ -o ./
#$ -l mem_free=50G

Trinity --seqType fq --max_memory 50G --normalize_reads --min_contig_length 300 --left %s  --right %s --CPU 15

	''' % (dir,F,R)

	filE = op.join(dir,"Assemble%s.sh" % dir)
	with open (filE,'w') as o:
    	o.write("%s" % text)


In [ ]:
for D in ./Trim*; do
	#echo "$D"
	cd "$D"
	qsub Assemble*
	cd ../
done

# Combine all assemblies and reduce redudancy and get annotations for final assembly

In [ ]:
find $PWD -name "Trinity.fasta" > AssembledGenomes

Modify this Assembled Genomes file and turn it into a bash script

In [ ]:
#!/usr/bin/env bash
#$ -N refineAssembly
#$ -S /bin/bash
#$ -V
#$ -j y
#$ -pe smp 20
#$ -cwd
#$ -e ./
#$ -o ./
#$ -l mem_free=50G



source activate exonerate
export fastanrdb=/home/menonm2/g/anaconda3/pkgs/exonerate-2.4.0-0/bin/fastanrdb
export PATH=$HOME/g/src/dDocent_run/cd-hit-v4.6.1-2012-08-27:$PATH

#List all the aseembled fasta here (copy the paths from the file above)

f1="../denovo16729/trinity_out_dir/Trinity.fasta"
f2="../denovo15092/trinity_out_dir/Trinity.fasta"
f3="../denovo14511/trinity_out_dir/Trinity.fasta"
f4="../denovo15797/trinity_out_dir/Trinity.fasta"
f5="../denovo7165/trinity_out_dir/Trinity.fasta"
f6="../denovo7365/trinity_out_dir/Trinity.fasta"
f7="../denovo7416/trinity_out_dir/Trinity.fasta"
f8="../denovo7714/trinity_out_dir/Trinity.fasta"
f9="../denovo8102/trinity_out_dir/Trinity.fasta"
f10="../denovo8168/trinity_out_dir/Trinity.fasta"
f11="../BSdenovo13353/trinity_out_dir/Trinity.fasta"
f12="../BSdenovo13283/trinity_out_dir/Trinity.fasta"
f13="../BSdenovo13733/trinity_out_dir/Trinity.fasta"
f14="../BSdenovo13508/trinity_out_dir/Trinity.fasta"
f15="../BSdenovo13779/trinity_out_dir/Trinity.fasta"
f16="../BSdenovo13736/trinity_out_dir/Trinity.fasta"
f17="../BSdenovo22442/trinity_out_dir/Trinity.fasta"
f18="../BSdenovo23524/trinity_out_dir/Trinity.fasta"
f19="../BSdenovo23762/trinity_out_dir/Trinity.fasta"
f20="../BSdenovo23832/trinity_out_dir/Trinity.fasta"

/home/menonm2/g/src/evigene19jan01/scripts/rnaseq/trformat.pl  -output combined.fasta -input $f1 $f2 $f3 $f4 $f5 $f6 $f7 $f8 $f9 $f10 $f11 $f12 $f13 $f14 $f15 $f16 $f17 $f18 $f19 $f20

~/g/src/evigene19jan01/scripts/prot/tr2aacds.pl -tidy -NCPU 10 -MAXMEM 131072 -log -cdna combined.fasta


$HOME/g/src/EnTAP-v0.9.1-beta/EnTAP --runP -i ../EviGene/okayset/combined.okay.aa --paths $HOME/g/src/EnTAP-v0.9.1-beta/entap_config.txt -d $HOME/eckertlab/databases/Diamond/bin/uniprot_sprot.dmnd -c fungi -c bacteria --taxon Pinus --qcoverage 70 --tcoverage 70 --out-dir $HOME/eckertlab/Mitra/chapter3/pipeline/denovo/byInd/EnTAP
